In [1]:
import sys
import time

sys.path.append('../..')

#load from local .env file
from dotenv import load_dotenv, find_dotenv
env = load_dotenv('.env', override=True)

#standard python
from typing import List, Dict, Tuple
import os

# external libraries
from tqdm import tqdm
from rich import print  # nice library that provides improved printing output (overrides default print function)

# external files
from src.reranker import ReRanker
from src.database.weaviate_interface_v4 import WeaviateWCS
from src.app_dev.app_functions import validate_token_threshold
from src.app_dev.query import (
    parse_context_results,
    CompletedQuery,
    CompletedQueryQueue,
    create_llm_prompt
)
from tiktoken import Encoding, get_encoding
from src.llm.prompt_templates import (
    question_answering_prompt_series,
    generate_prompt_series,
    huberman_system_message
)
from src.llm.llm_utils import load_azure_openai

/usr/local/lib/python3.10/site-packages/pydantic/_internal/_config.py:284: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.10/site-packages/litellm/proxy/_types.py:167: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 style `@model_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  @root_validator(pre=True)
/usr/local/lib/python3.10/site-packages/litellm/proxy/_types.py:254: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2

In [2]:
#read env vars from local .env file
api_key = os.environ['WEAVIATE_API_KEY']
url = os.environ['WEAVIATE_ENDPOINT']

#instantiate client
client = WeaviateWCS(url, api_key)

In [3]:
#display available collection names on cluster
client.show_all_collections()

['Huberman_subset_minilm_test',
 'Huberman_minilm_128',
 'Huberman_minilm_256',
 'Huberman_minilm_512']

In [4]:
#set collection name to run queries on
collection_name = 'Huberman_minilm_256'

In [5]:
#Initial query
query = 'What episode talks the most about hypertrophy training?'

In [6]:
alpha = 0.45
hyb_response = client.hybrid_search(query, collection_name, alpha=alpha, limit=5)

In [7]:
reranker = ReRanker()
reranked_results = reranker.rerank(
            hyb_response, query, apply_sigmoid=False, top_k=5
        )

/usr/local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
encoding = get_encoding("cl100k_base")

valid_results = validate_token_threshold(
            reranked_results,
            question_answering_prompt_series,
            query=query,
            tokenizer=encoding,
            token_threshold=4000,
            content_field="content"
        )

In [9]:
#Getting this here so have for the initial LLM call that we currently use.
context_series = generate_prompt_series(query, valid_results, 1)

In [10]:
llm = load_azure_openai('gpt-35-turbo')

In [11]:
llm_response = llm.chat_completion(huberman_system_message,
    user_message=context_series,
    temperature=0.5,
    max_tokens=1000)
    
print(llm_response)

The episode that talks the most about hypertrophy training is the one featuring Dr. Andy Galpin as the guest. In 
this episode, Dr. Galpin discusses the ways to induce hypertrophy and the stimuli needed to promote muscle growth. 
He emphasizes the importance of resistance training for hypertrophy and clarifies the factors, including hormonal 
and nutritional aspects, that play a role in inducing hypertrophy. Additionally, he describes hypertrophy training 
as "idiot proof" in terms of programming, highlighting its importance for both men and women.

In [12]:
# For now, I just want title, guest, and content.
context_results = parse_context_results(valid_results)
# Create a CompletedQuery object
completed_query = CompletedQuery(query, context_results, llm_response)

# Add the CompletedQuery object to the CompletedQueryQueue
# This is how we'll store the last 5 queries.
completed_query_queue = CompletedQueryQueue()
completed_query_queue.add_query(completed_query)

In [20]:
# This is going to be my vague follow-up query.
new_user_query = "Who else discusses that topic?"

# We need to convert the data structure holding the last 5 queries to a string
# so we can send it to the llm.
completed_query_string_list = completed_query_queue.to_string()

# Combine the new vague query with the previous queries and their contexts, and 
# answers to create a new llm prompt.
query_rewrite_prompt = create_llm_prompt(new_user_query, completed_query_string_list)
print(query_rewrite_prompt)

New User Query: "Who else discusses that topic?"

Query History:
User Query: What episode talks the most about hypertrophy training?, Context Results: [Title: Dr. Andy Galpin: 
Optimal Nutrition & Supplementation for Fitness | Huberman Lab Guest Series, Guest: Dr. Andy Galpin, Content: The 
second episode, you educated us about strength, speed and hypertrophy training. And there too, an immense amount of
incredible knowledge. We got way down into the details. You explained sets, reps, the rationale for sets, reps, 
rest, cadence, number of workouts per week. I'm definitely going to take away my need to do some speed-based 
training and some power-based training. Normally, I think in terms of strength or hypertrophy. And I'm relieved to 
learn that a lot of the speed and power-based training, it's low intensity enough that it can be done fairly often 
and incorporated into my program, which already touches on strength and hypertrophy and indeed some endurance work 
as well. So amazing tips that you provided there. I'm certainly going to implement the three to five program that 
you described, three to five exercises done for three to five repetitions, three to five minutes between sets. 
You're doing this three to five times per week and so on and so forth. All the details, again, timestamped in the 
strength and hypertrophy episode show notes. So just incredible. We even use that to set up PR with you this week. 
That's right., Title: Dr. Andy Galpin: Optimize Your Training Program for Fitness & Longevity | Huberman Lab Guest 
Series, Guest: Dr. Andy Galpin, Content: Now we've got a little bit of distinction. If you're going to train 
strength and hypertrophy, as we talked about in that episode, at the base, those are going to be complementary. You
add on some muscle, you're going to get stronger. You start training for strength, it's probably going to help you 
add on some muscle mass. As you get to the end of that spectrum, the overlap between the two starts to go away, 
such that if you truly wanted to maximize strength above everything else, if you continue to train for hypertrophy 
as well, that's going to take too many resources out of your recovery bin, and you won't be able to do that. The 
inverse would also be the same, right? If you're training to maximize strength, you wouldn't be able to put enough 
volume on to get sufficient hypertrophy. So if you wanted to then combine speed with hypertrophy, you're going 
farther away from each other, which means it's going to be more and more distraction. So the hypertrophy training 
would cause a ton of fatigue. You wouldn't be able to go at max speed for your speed or power strength., Title: The
Science of Vision, Eye Health & Seeing Better, Guest: Andrew Huberman, Content: We are now beginning a new topic 
for the next four to five episodes of the Huberman Lab Podcast. Before we move into that, I want to just briefly 
touch on a couple of questions that I got from the last episode, which was related to the science of endurance 
training. I described the four kinds of endurance training. We posted protocols of the specific four kinds of 
endurance training at hubermanlab.com. Just go to that episode. You can see the download. It's a zero cost PDF. I 
got a lot of questions about what's called concurrent training, which is how to program endurance training if you 
are also interested in strength and hypertrophy training, or how to incorporate strength and hypertrophy training, 
which was in the previous episode, with endurance training. This can all be made very simple. Ask yourself, what 
are you trying to emphasize? And then emphasize that for a 10 to 12 week cycle. So if you're mostly interested in 
endurance, I would say use a three to two ratio, maybe get three endurance training workouts per week, maybe four 
and two strength and hypertrophy workouts., Title: Dr. Andy Galpin: Optimal Protocols to Build Strength & Grow 
Muscles | Huberman Lab Guest Series, Guest: Dr. An

In [21]:
# This is what we'd get currently.
llm_response = llm.chat_completion(huberman_system_message,
    user_message=new_user_query,
    temperature=0.5,
    max_tokens=1000)

print(llm_response)

The Huberman Lab podcast is hosted by Dr. Andrew Huberman, a neuroscientist and professor at Stanford School of 
Medicine. While there are other podcasts and platforms that discuss neuroscience and science-based tools, I can 
only provide information about the Huberman Lab podcast.

In [22]:
# I was using this system message but haven't tried using the old one. 

start_time = time.time()

# This is what the new prompts get with the vague question.
llm_response = llm.chat_completion(huberman_system_message,
    user_message=query_rewrite_prompt,
    temperature=0.5,
    max_tokens=1000)

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

print(llm_response)

Execution time: 0.6103546619415283 seconds

What other podcasts or episodes discuss hypertrophy training?